In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import cv2 
import os
import sys
import re
import random
import xml.etree.ElementTree as ET
import albumentations as A
import random
from sklearn.cluster import KMeans
%matplotlib inline

In [ ]:
def count_class_distribution(path):
    classes = {}
    root_path = os.getcwd()

    def plot(data):
        labels, counts = zip(*data.items())
        plt.figure(figsize=(6, 4))
        bars = plt.bar(labels, counts, color='green')
        plt.xlabel('Class Labels')
        plt.ylabel('Count')
        plt.title('Class Distribution')
        for bar, count in zip(bars, counts):
            plt.text(bar.get_x() + bar.get_width() / 2 - 0.1, bar.get_height() + 0, str(count), ha='center', va='bottom')
        plt.tight_layout()
        plt.show()

    if os.path.exists(os.path.join(root_path, path)):
        for root_dir, dirs, files in os.walk(os.path.join(os.path.join(root_path, path))):
            for filename in files:
                xml_path = os.path.join(root_dir, filename)
                if xml_path.endswith(".xml"):
                    tree = ET.parse(xml_path)
                    root_xml = tree.getroot()

                    for obj in root_xml.findall('.//object'):
                        class_label = obj.find('name').text

                        if class_label in classes:
                            classes[class_label] += 1
                        else:
                            classes[class_label] = 1
        plot(classes)
    else:
        print("Path does not exist")

count_class_distribution("")

In [ ]:
count_class_distribution("")

In [ ]:
def xml_dataFrame(annots_path, images_path= None):
    root_path = os.getcwd()
    def read_files(ap):
        data = []
        for root, dirs, files in os.walk(os.path.join(root_path, ap)):
            for i in files:
                data.append(
                    os.path.join(root, i)
                )
        return data
    def search_image_path(folder_name):
        data = {}
        for root, dirs, files in os.walk(os.path.join(root_path, folder_name)):
            for i in files:
                data[i.split(".")[0]] = os.path.join(root, i)
        return data

    image_data = search_image_path(images_path)
    data_dic = {"image_path":[],"annot_path": [], "class_label": []}
    
    if os.path.exists(os.path.join(root_path, annots_path)):
        data = read_files(annots_path)
        for i in data:
            if i.endswith(".xml"):
                name = re.findall(re.compile(r"[a-zA-Z-0-9\._]+"), i)[-1]
                tree = ET.parse(i)
                root_xml = tree.getroot()
                for obj in root_xml.findall('.//object'):
                    an = name.split('.')[0]
                    class_label = obj.find('name').text
                    data_dic['image_path'].append(image_data[an])
                    data_dic['annot_path'].append(i)
                    data_dic['class_label'].append(class_label)
        return pd.DataFrame(
            {
                "image":data_dic['image_path'],"annot": data_dic['annot_path'], "Class": data_dic['class_label']
            }
        )
    else:
        print("Path does not exist")

In [ ]:
train_df = xml_dataFrame("","")

In [ ]:
tm = train_df[train_df["Class"] == "tm"]
tnb = train_df[train_df["Class"] == "tnb"]
other = train_df[train_df["Class"] == "other"]

In [ ]:
def get_only_class_df(df):
    idx = []
    annot = df['annot'].values
    for i,v in enumerate(annot):
        tree = ET.parse(v)
        root_xml = tree.getroot()
        temp = []
        for obj in root_xml.findall('.//object'):
            class_label = obj.find('name').text
            temp.append(class_label)
        if len(temp) == 1:
            idx.append(i)
    return df.iloc[idx,:]

# Data Augmentation

In [ ]:

def augment_image_with_xml(image_path: str, xml_path: str, ip: str, ap: str, c: int, class_name: str, rm: list):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Find and remove objects with name
    for obj in root.findall('object'):
        name = obj.find('name').text
        if name in rm:
            root.remove(obj)

    bounding_boxes = []
    for obj in root.findall('.//object'):
        xmin = int(obj.find('bndbox/xmin').text)
        ymin = int(obj.find('bndbox/ymin').text)
        xmax = int(obj.find('bndbox/xmax').text)
        ymax = int(obj.find('bndbox/ymax').text)
        bounding_boxes.append([xmin, ymin, xmax, ymax])

    bbox_params = A.BboxParams(
        format='pascal_voc', min_area=0, min_visibility=0, label_fields=['category_id'])
    rotation = random.choice([(-10, 10), (-12, 12), (-5, 5)])
    augmentation = A.Compose([
        A.HorizontalFlip(p=0.8),
        A.Rotate(limit=rotation, border_mode=4,
                 value=None, mask_value=None, always_apply=False, p=0.9),
        A.ISONoise(color_shift=(0.01, 0.02), intensity=(0.1, 0.2))
    ], bbox_params=bbox_params)

    augmented = augmentation(image=image, bboxes=bounding_boxes, category_id=[
                             1]*len(bounding_boxes))
    augmented_image = augmented['image']
    augmented_boxes = augmented['bboxes']

    for obj, new_bbox in zip(root.findall('.//object'), augmented_boxes):
        obj.find('bndbox/xmin').text = str(int(new_bbox[0]))
        obj.find('bndbox/ymin').text = str(int(new_bbox[1]))
        obj.find('bndbox/xmax').text = str(int(new_bbox[2]))
        obj.find('bndbox/ymax').text = str(int(new_bbox[3]))

    image_name = re.findall(re.compile(r"[a-zA-Z-0-9\._]+"), image_path)
    imgName = image_name[-1].split('.')
    imgName = imgName[0] + "_AUG" + class_name + str(c) + '.'+imgName[1]
    image_name[-1] = imgName
    image_name = os.path.join(*image_name)

    xmal_name = re.findall(re.compile(r"[a-zA-Z-0-9\._]+"), xml_path)
    xmalName = xmal_name[-1].split('.')
    xmalName = xmalName[0] + "_AUG" + class_name + str(c) + '.'+xmalName[1]
    xmal_name[-1] = xmalName
    xmal_name = os.path.join(*xmal_name)

    output_image_path = os.path.join(ip, os.path.basename(image_name))
    output_xml_path = os.path.join(ap, os.path.basename(xmal_name))
    cv2.imwrite(output_image_path, cv2.cvtColor(
        augmented_image, cv2.COLOR_RGB2BGR))
    tree.write(output_xml_path)


def apply_augment(df, image_save_path, annot_save_path, loop, class_name, rm_class):
    df = df.reset_index().drop('index', axis=1)
    root_path = os.getcwd()
    if os.path.exists(os.path.join(root_path, image_save_path)) and os.path.exists(os.path.join(root_path, annot_save_path)):
        for i in range(df.shape[0]):
            data_ = df.iloc[i, :]
            image_path = data_['image']
            xml_path = data_['annot']
            for j in range(loop):
                augment_image_with_xml(
                    image_path, xml_path, image_save_path, annot_save_path, j, class_name, rm_class)

        print("annotation done")

In [ ]:
def remove_augmentaed_data(path):
    root_path = os.path.join(os.getcwd(), path)
    for root, dirs, files in os.walk(root_path):
        for i in files:
            if "AUG" in i:
                os.remove(os.path.join(root, i))

In [ ]:
def draw_bounding_boxes(image_path, xml_path, output_path=None):
    # Load image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Load XML annotations
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Draw bounding boxes on the image
    for obj in root.findall('.//object'):
        xmin = int(obj.find('bndbox/xmin').text)
        ymin = int(obj.find('bndbox/ymin').text)
        xmax = int(obj.find('bndbox/xmax').text)
        ymax = int(obj.find('bndbox/ymax').text)

        # Draw rectangle
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    plt.imshow(image)

# Example usage
image_path = ''
xml_path = ''
output_path = None  # Optional: provide an output path to save the image

draw_bounding_boxes(image_path, xml_path, output_path)


In [ ]:
def change_xml_filename(path, image_path):
    '''
        path: xml folder location
        image_path: image path, to be written in xml's <filename>  (relative path only)
    '''
    images = []
    for root, dirs, files in os.walk(os.path.join(os.getcwd(), image_path)):
        for i in files:
            images.append(i)
    root_path = os.getcwd()
    if os.path.exists(os.path.join(root_path, path)):
        for root_dir, dirs, files in os.walk(os.path.join(root_path, path)):
            for i in files:
                if i.endswith(".xml"):
                    xml_file_path = os.path.join(root_dir, i)
                    tree = ET.parse(xml_file_path)
                    root_xml = tree.getroot()  # Change the variable name to avoid conflict
                    filename_element = root_xml.find('filename')
                    if filename_element is not None:
                        n = i.split('.')[0]
                        ext = [i for i in images if n in i][0].split('.')[-1]
                        n = i.split('.')[0] + '.' +ext
                        filename_element.text = n
                    else:
                        print("Element 'filename' not found in the XML file.")
                    tree.write(xml_file_path)

In [2]:
def change_xml_path(path, image_path):
    '''
        path: xml folder location
        image_path: image path, to be written in xml's <path>  (relative path only)
    '''
    root_path = os.getcwd()
    if os.path.exists(os.path.join(root_path, path)):
        for root_dir, dirs, files in os.walk(os.path.join(root_path, path)):
            for i in files:
                if i.endswith(".xml"):
                    xml_file_path = os.path.join(root_dir, i)
                    tree = ET.parse(xml_file_path)
                    root_xml = tree.getroot()  # Change the variable name to avoid conflict
                    path_element = root_xml.find('path')
                    filename_element = root_xml.find('filename')
                    if path_element is not None:
                        image_path_ = os.path.join(os.getcwd(),image_path, filename_element.text)
                        path_element.text = image_path_
                    else:
                        print("Element 'filename' not found in the XML file.")
                    tree.write(xml_file_path)

In [2]:
from pascal import annotation_from_xml
def xml_to_yolo(path, to_save):
    
    label_map = {"tm": 0, "tnb": 1, "other": 2}
    
    root_path = os.getcwd()
    if os.path.exists(os.path.join(root_path, path)):
        for root_dir, dirs, files in os.walk(os.path.join(root_path, path)):
            for i in files:
                n = i.split('.')[0] + '.txt'
                file = os.path.join(root_dir, i)
                ann = annotation_from_xml(file)
                yolo_ann = ann.to_yolo(label_map)
                with open(os.path.join(root_path, to_save, n), "w") as f:
                    f.write(yolo_ann)